In [1]:
import re
from collections import Counter, defaultdict
from functools import cache
from itertools import combinations, cycle, zip_longest
from typing import *
from dataclasses import dataclass

import black
import jupyter_black
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def first(iterable):
    return next(iter(iterable))

In [25]:
# Day 1: Chronal Calibration
def first_repeat(frequencies):
    freqs = cycle(frequencies)
    found = set()
    current = 0
    while current not in found:
        found.add(current)
        current += next(freqs)
    return current


lines = open("2018/1.txt").read().splitlines()
frequencies = [int(line) for line in lines]
print(f"Part 1: {sum(frequencies)}")  # 411
print(f"Part 2: {first_repeat(frequencies)}")  # 56360

Part 1: 411
Part 2: 56360


In [87]:
# Day 2: Inventory Management System
def checksum(ids):
    def contains(text, repeated):
        "`text` contains exactly `repeated` repeated characters"
        return repeated in Counter(text).values()

    twos = sum(contains(id, 2) for id in ids)
    threes = sum(contains(id, 3) for id in ids)
    return twos * threes


def common_letters(ids):
    "Common letters for the two ids that only differ by one character."
    for a_, b_ in combinations(ids, 2):
        common = [a for a, b in zip(a_, b_) if a == b]
        if len(a_) - len(common) == 1:
            return "".join(common)


ids = open("2018/2.txt").read().splitlines()
print(f"Part 1: {checksum(ids)}")  # 5166
print(f"Part 2: {common_letters(ids)}")  # cypueihajytordkgzxfqplbwn

Part 1: 5166
Part 2: cypueihajytordkgzxfqplbwn


In [120]:
# Day 3: No Matter How You Slice It
def patch(row, column, width, height):
    "Return a set with all coordinates"
    return {
        (r, c) for c in range(column, column + width) for r in range(row, row + height)
    }


def point_claims(claims):
    "Number of claims per point"
    patches = Counter()
    for _, column, row, width, height in claims:
        patches.update(patch(row, column, width, height))
    return patches


def overlapping_inches(claims):
    return sum(p > 1 for p in point_claims(claims).values())


def intact_claim_id(claims):
    overlapping = {
        coord for coord, overlaps in point_claims(claims).items() if overlaps > 1
    }

    for id, column, row, width, height in claims:
        claim = patch(row, column, width, height)
        if all(point not in overlapping for point in claim):
            return id


# ID, column, row, width, height
lines = open("2018/3.txt").read().splitlines()
claims = [tuple(int(x) for x in re.findall(r"\d+", line)) for line in lines]
print(f"Part 1: {overlapping_inches(claims)}")  # 115348
print(f"Part 2: {intact_claim_id(claims)}")  # 188

Part 1: 115348
Part 2: 188


In [105]:
# Day 4: Repose Record
def sleep_schedule(lines: List[str]) -> DefaultDict[int, Counter]:
    log: List[List[str]] = sorted([line[1:].split("] ") for line in lines])
    guard = snooze = 0
    schedule = defaultdict(Counter)
    for timestamp, entry in log:
        if p := parse("Guard #{:d} begins shift", entry):
            guard = p[0]
        elif p := parse("falls asleep", entry):
            snooze = int(timestamp[-2:])
        elif p := parse("wakes up", entry):
            wakes = int(timestamp[-2:])
            assert wakes > snooze
            schedule[guard].update(range(snooze, wakes))
        else:
            raise ValueError("Can't parse", entry)
    return schedule


def part1(schedule: DefaultDict[int, Counter]) -> int:
    """
    Find the guard that has the most minutes asleep. What minute does that guard spend
    asleep the most?
    """
    guard, _ = sorted(
        [(guard, slept.total()) for guard, slept in schedule.items()],
        key=lambda x: x[1],
    )[-1]
    minute = schedule[guard].most_common(1)[0][0]

    return guard * minute


def part2(schedule: DefaultDict[int, Counter]) -> int:
    """
    Of all guards, which guard is most frequently asleep on the same minute?
    """
    guard, (minute, _) = sorted(
        [(guard, slept.most_common(1)[0]) for guard, slept in schedule.items()],
        key=lambda x: x[1][1],
    )[-1]

    return guard * minute


lines: str = open("2018/4.txt").read().splitlines()
schedule: DefaultDict[int, Counter] = sleep_schedule(lines)
print(f"Part 1: {part1(schedule)}")  # 35184
print(f"Part 2: {part2(schedule)}")  # 37886

Part 1: 35184
Part 2: 37886


In [270]:
# Day 5: Alchemical Reduction
def one_pass(polymers):
    result = []
    chars = zip_longest(polymers, polymers[1:], fillvalue=polymers[-1])
    for cur, nxt in chars:
        if cur == nxt.swapcase():
            next(chars)
        else:
            result.append(cur)
    return result


def reduce_all(text):
    polymers = list(text)
    length = 0
    while length != len(polymers):
        length = len(polymers)
        polymers = one_pass(polymers)
    return len(polymers)


def remove_char(char, text):
    table = {ord(char.lower()): None, ord(char.upper()): None}
    return text.translate(table)


def shortest_polymer(text):
    units = set(text.lower())
    lengths = {unit: reduce_all((remove_char(unit, text))) for unit in units}
    return min(lengths.values())


text = open("2018/5.txt").read().strip()
print(f"Part 1: {reduce_all(text)}")  # 9704, 3.5s runtime
print(f"Part 2: {shortest_polymer(text)}")  # 6942, 1m 10s

# Only use one pass, see
# https://www.reddit.com/r/adventofcode/comments/a3912m/comment/eb4ff0j/?utm_source=reddit&utm_medium=web2x&context=3 
def reduce_all(text):
    buf = []
    for c in text:
        if buf and c == buf[-1].swapcase():
            buf.pop()
        else:
            buf.append(c)
    return len(buf)


# shortest_polymer(text) # Finishes in 0.3s.

Part 1: 9704
Part 2: 6942


In [111]:
# Day 6: Chronal Coordinates
def min_coords(coords):
    return min(coord[0] for coord in coords), min(coord[1] for coord in coords)


def max_coords(coords):
    return max(coord[0] for coord in coords), max(coord[1] for coord in coords)


def points(coords):
    min_x, min_y = min_coords(coords)
    max_x, max_y = max_coords(coords)
    return {(x, y) for x in range(min_x, max_x + 1) for y in range(min_y, max_y + 1)}


def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


@cache
def closest_coord(point, coords):
    "Return closest coordinate for a point, None if there is a tie."
    distances = {(coord): manhattan_distance(point, coord) for coord in coords}
    shortest_distance = min(distances.values())
    closest = [
        coord for coord, distance in distances.items() if distance == shortest_distance
    ]
    if len(closest) == 1:
        return first(closest)


def safe_coords(coords):
    def bounding_box(coords):
        min_x, min_y = min_coords(coords)
        max_x, max_y = max_coords(coords)
        top_bottom = {(y, x) for x in range(min_x, max_x + 1) for y in (min_y, max_y)}
        left_right = {(y, x) for y in range(min_y, max_y + 1) for x in (min_x, max_x)}
        return top_bottom | left_right

    # Infinite coords are the ones that are closest to any point in the bounding box
    infinite = {
        closest_coord(point, coords)
        for point in bounding_box(coords)
        if closest_coord(point, coords)
    }
    return set(coords) - infinite


def largest_area(coords):
    safe = safe_coords(coords)
    areas = Counter(
        closest_coord(point, coords)
        for point in points(coords)
        if closest_coord(point, coords) in safe
    )
    return first(areas.most_common(1))[1]


def distance_lt(coords, value=10_000):
    def distance_to_all(point, coords):
        return sum(manhattan_distance(point, coord) for coord in coords)

    return sum(distance_to_all(point, coords) < value for point in points(coords))


data = open("2018/6.txt").read().splitlines()
coords = tuple(tuple(int(x) for x in line.split(", ")) for line in data)
print("Part 1:", largest_area(coords))  # 2906
print("Part 2:", distance_lt(coords))  # 50530

Part 1: 2906
Part 2: 50530


In [192]:
# Day 7: The Sum of Its Parts
def starting_steps():
    children = {x for value in steps.values() for x in value}
    parents = {x for x in steps.keys()}
    return list(parents - children)


def path():
    neighbors = starting_steps()
    path = []

    while neighbors:
        neighbors.sort(reverse=True)
        current = neighbors.pop()
        path.append(current)
        neighbors.extend(
            child
            for child in steps[current]
            if all(parent in path for parent in needs[child])
        )

    return "".join(path)


@dataclass
class Worker:
    step: str = ""
    time_remaining: int = 0

    def add_step(self, step: str, offset=61):
        new_time = ord(step) - ord("A") + offset
        self.time_remaining = new_time
        self.step = step

    def tick(self):
        self.time_remaining -= 1

    def available(self):
        return self.time_remaining <= 0

    def completed(self):
        return self.time_remaining == 0


def complete():
    neighbors = starting_steps()
    done = set()
    workers = [Worker() for _ in range(5)]

    for second in range(10**4):
        # Check who is done and add new possible neighbors
        for worker in workers:
            worker.tick()
            if worker.completed():
                done.add(worker.step)
                neighbors.extend(
                    child
                    for child in steps[worker.step]
                    if all(parent in done for parent in needs[child])
                )
        neighbors.sort(reverse=True)
        # Assign all possible steps to workers
        for worker in workers:
            if worker.available() and neighbors:
                worker.add_step(neighbors.pop())
        # Check if we are done
        if all(w.available() for w in workers):
            return second


lines = open("2018/7.txt").read().splitlines()
steps: DefaultDict[str, List[str]] = defaultdict(list)
needs: DefaultDict[str, List[str]] = defaultdict(list)

for line in lines:
    parent, child = parse("Step {} must be finished before step {} can begin.", line)
    steps[parent].append(child)
    needs[child].append(parent)

print(f"Part 1: {path()}")  # EFHLMTKQBWAPGIVXSZJRDUYONC
print(f"Part 2: {complete()}")  # 1056

Part 1: EFHLMTKQBWAPGIVXSZJRDUYONC
Part 2: 1056


In [154]:
[Worker()] * 5

[Worker(step='', time_remaining=0),
 Worker(step='', time_remaining=0),
 Worker(step='', time_remaining=0),
 Worker(step='', time_remaining=0),
 Worker(step='', time_remaining=0)]